# Nemweb file download, split, aggregation and converstion

This script downloads data files from https://www.nemweb.com.au/REPORTS/ and https://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/ .

1. Crawl those pages to find a list of URLs for CSV and .zip files
2. Filter that list. e.g. delete large files we don't need
3. Download those files. (Suprisingly hard. The server is unreliable. Throttles us, gives corrupt files etc)
4. Unzip each zip. (Possibly recursively. They have zips of zips of CSVs). The total dataset is about 1TB uncompressed. So we keep .csv files as .csv.gz (gzip compressed CSV)
5. For each CSV file, split it up into several CSV files. This is because AEMO's CSV files contain data for different tables (e.g. pricing and billing stuff in the same file.)
6. After splitting into groups of files, figure out which 'table name' to map each file to a table [here](https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report.htm)

The code may seem more complex than you expected. That's mostly because I want to do multiprocessing to speed it up. But then this complicates things because:

* our downloads get throttled, so we need to sleep and retry
* Sometimes you can download/unzip/process 1000 files, and then file 1001 fails. I want to keep processing the remainder, gather up all the success/failures, and then compare them. So there's some try/catch stuff to handle that.

## Remaining to do (as of 30/11/2023)

* debug why mwe.csv can't be processed with pyarrow. Continue shrinking down to mwe.
* try doing the parquet converstion with R
* try re-running the cell at the bottom to test which rows to skip
* If I can't solve it, for the parquet conversion, only choose the tables we care about. (Because some fail with an error, and I don't know why.)
* Re-run the download for all files.


In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
from multiprocessing import Pool
import urllib3
import shutil
from urllib.parse import  urljoin, urlparse
from zipfile import ZipFile
from time import sleep, time
import re
import gzip
from random import randrange, shuffle
from uuid import uuid4
from typing import Set, Tuple # type annotations, for doc clarity only. Ignore them if you're not used to them
from itertools import zip_longest
import csv
from copy import deepcopy as copy
import pickle
import datetime as dt
from typing import Dict

from tqdm import tqdm # progress bar animation
#from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup # webscraping
import pandas as pd
import pyarrow
import pyarrow.csv
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyarrow.parquet
import numpy as np


## Constants and Configuration

In [ ]:
start_urls = [
    'https://www.nemweb.com.au/REPORTS/CURRENT/', # last year
    #'https://www.nemweb.com.au/REPORTS/ARCHIVE/', # last year
    'https://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2023/MMSDM_2023_10/', 
    #'https://nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/' # 1-10 years old, in a different folder and zip structure
]
file_suffixes = ['.zip', '.csv'] # will be made case insensitive later
expected_domains = ('www.nemweb.com.au', 'nemweb.com.au')

# ignore any URL containing one of these
url_substrings_to_skip = [
    '/NEMDE/', # don't bother looking in these folders
    '/Gas_Supply_Guarantee/',
    '/VicGas/',
    '/DWGM/',
    'Predispatch', # predictions (very large)
    'FCAS', # unrelated to our purposes, and quite large
    'BIDPEROFFER', # large, unrelated
    'BIDDAYOFFER', # large, unrelated
    'BIDOFFER',
    'DAYOFFER',
    'MTPASA_OFFER',
    'MTPASA',
    'PDPASA',
    'PRUDENTIAL',
    'MMSDM_CLI_', # not data
    'MMSDM_GUI_', # not data
]

max_files_per_page = 2

base_data_dir = 'data'
urls_file_path = os.path.join(base_data_dir, 'urls.txt')
raw_files_path = os.path.join(base_data_dir, '01-A-raw/')
unzipping_temp_path = os.path.join(base_data_dir, '01-B-unzipping/')
unzipped_files_path = os.path.join(base_data_dir, '01-C-unzipped/')
split_unmapped_files_path = os.path.join(base_data_dir, '01-D-split/')
split_mapped_files_path = os.path.join(base_data_dir, '01-E-split-mapped/')
one_parquet_per_table_path = os.path.join(base_data_dir, '01-F-one-parquet-per-table/')
debug_path = os.path.join(base_data_dir, 'debug/')


compresslevel = 2 # how hardcore the gzip algorithm is for .csv.gz files. Low to prioritise speed over size

# process files with multiprocessing, to be faster.
# If set to False, will use a for loop, which gives clearer traceback error messages.
use_multiprocessing = True

# set to True to leave one unused CPU when multiprocessing
# so that you can still do other stuff on your laptop without your internet browser or whatever being laggy
leave_unused_cpu = True

In [ ]:
metadata_path = os.path.join(base_data_dir, 'MMS_Data_Model_Table_to_File_to_Report_Relationships_51.xlsx')
sheet = 'MMSDM_v5-1'
report_subtype_column = 'PDR_REPORT_SUB_TYPE'
report_name_column = 'PDR_REPORT_NAME'
table_name_column = 'MMSDM_TABLE_NAME'

REPORT_NAME_NULL_PLACEHOLDER = 'NULL'
REPORT_SUBTYPE_NULL_PLACEHOLDER = 'NULL'

# AEMO data is always in UTC+10
TIMEZONE_OFFSET=10

In [ ]:
# on Linux and Mac this makes this python process lower priority
# so when it's running and using up all your CPU, your interface won't lag.
# So you can keep browsing the web, typing documents etc
try:
    os.nice(20)
except OSError: # ignore error, this is probably on Windows
    pass

## Logging

Each step takes a long time (many files). I want to know that it's doing something. But if I just `print()`, there will be too much text. So I write to a file, and you can look at that file to watch the logs. (On Linux or Mac use `tail -f log.txt` in a terminal. Windows users probably have to buy some app from the Windows app store. I dunno.)

As a lazy but concurrency-safe approach, we just open and close the file for each message. Meh, good enough for now. (The `logging` library is not multiprocessing safe. You can [make it work with logs of effort](https://docs.python.org/3/howto/logging-cookbook.html#logging-to-a-single-file-from-multiple-processes))

For this approach, log with:

* `logger.info("Something")`
* `logger.error("Something")`

And to wipe the file (e.g. when retrying a cell) call `logger.reset()`.

In [ ]:
class Logger:
    def __init__(self, path=os.path.join(base_data_dir, 'log.txt')):
        self.path = path
        self.reset()
        
    def write(self, msg):
        with open(self.path, 'a') as f:
            f.write(msg.rstrip() + '\n')
    def info(self, msg):
        self.write(f"INFO: {msg}")
    def warn(self, msg):
        self.write(f"WARNING: {msg}")
    def warning(self, msg):
        self.warn(msg)
    def error(self, msg):
        self.write(f"ERROR: {msg}")

    # erase the file
    # but leave a blank file in place
    # to do that, open in `w` mode, and write nothing.
    def reset(self):
        with open(self.path, 'w'):
            pass

logger = Logger()
logger.info("Initialising Logger")

## Discover which files to download

We don't actually download the .zip or .csv files yet.
We just get a list of the URLs of the .zip and .csv files.

In [ ]:

# create a session object to re-use between requests
# to hopefully speed up downloads by not re-doing the TLS handshake for each HTTP request
# (unsure if this actually speeds things up)
session = requests.Session()

In [ ]:
urls = []

progress_bar = tqdm(leave=False)


def force_https(url):
    p = urlparse(url)
    if p.scheme != 'https':
        assert url.count('http://') == 1, f"Strange URL: {url}" # abort, for security reasons
        url = url.replace('http://', 'https://')
    return url

def is_child_of(parent, child):
    p = urlparse(parent)
    c = urlparse(child)
    return (p.hostname in expected_domains) and (c.hostname in expected_domains) \
        and c.path.rstrip('/').startswith(p.path.rstrip('/')) \
        and c.path.rstrip('/') != p.path.rstrip('/')

# unit tests
assert is_child_of('https://www.nemweb.com.au/REPORTS/', 'https://www.nemweb.com.au/REPORTS/CURRENT/')
assert not is_child_of('https://www.nemweb.com.au/REPORTS/CURRENT', 'https://www.nemweb.com.au/REPORTS/')
assert not is_child_of('https://www.nemweb.com.au/REPORTS/CURRENT/', 'https://www.nemweb.com.au/REPORTS/CURRENT/')
assert not is_child_of('https://www.nemweb.com.au/REPORTS/CURRENT/', 'https://www.nemweb.com.au/REPORTS/ARCHIVE/')

def extract_links(url):
    logger.info(f"Checking for links in {url}")
    if any(ss in url for ss in url_substrings_to_skip):
        # skip this. Don't bother looking inside it.
        logger.info(f"Skipping {url} because of substrings")
        return []
    # this is a web page listing other files or pages of other files
    r = session.get(url)
    r.raise_for_status()
    html = r.text

    # it's called "soup" because the python webscraping library is called "beautiful soup"
    soup = BeautifulSoup(html)
    links = [a['href'] for a in soup.find_all('a')]
    
    # convert potentially relative URLs to absolute
    links = [urljoin(start_url, u) for u in links]

    # remove any links that aren't files on this page or subpages
    # watch out, the domain can sometimes change (www. removed or added)
    links = [u for u in links if is_child_of(parent=url, child=u)]

    # recursively check child pages
    links_to_children = [u for u in links if u.endswith('/')]
    links_of_children = [extract_links(u) for u in links_to_children]
    # flatten list of list to just a list
    links_of_children = [u for links_of_child in links_of_children for u in links_of_child]

    # filter to only ones matching our file suffixes
    links = [u for u in links if any(u.lower().endswith(ext.lower()) for ext in file_suffixes)]

    # delete any links which contain the blacklisted substrings
    links = [u for u in links if not any(ss in url for ss in url_substrings_to_skip)]
    
    # only a few files per page
    # except MMSDM pages, which have one file of each type
    if max_files_per_page and (max_files_per_page > 0) and ('/MMSDM/' not in url):
        links = links[:max_files_per_page]
    links.extend(links_of_children)

    progress_bar.update()
    logger.info(f"Found {len(links)} links at/under {url}")
    return links

urls = []
for start_url in start_urls:
    urls.extend(extract_links(start_url))

progress_bar.close()
print(f"Discovered {len(urls)} URLs")

In [ ]:
print(f"Writing {len(urls)} urls to {urls_file_path}")
with open(urls_file_path, 'w') as f:
    for u in urls:
        f.write(u + '\n')

In [ ]:
# if you want to skip re-indexing, run the playbook from here
with open(urls_file_path, 'r') as f:
    urls = [line.strip() for line in f]

Now we filter the list of URLs, to discard the ones we don't care about. (Large ones, gas ones etc)

Just delete URLs with string from `url_substrings_to_skip` in them.

In [ ]:
urls = [u for u in urls if not any(s.lower() in u.lower() for s in url_substrings_to_skip)]
len(urls)

In [ ]:
# For the older files, each year is available as one big file, or lots of little files
# e.g. https://nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2017/
# We don't want to download both
# drop the few large files, keep the little ones
urls = [u for u in  urls if not re.search(r"MMSDM_\d{4}_\d{2}.zip", u)]

## Download the files


Since we're downloading a lot of files, from the other side of the world, over slow wifi, we can get an error.
We don't want to redownload all the files when one has an error. So let's just carry on, then retry the whole lot.
When retrying, check if the file already exists on disk. If not, dowload it.  If yes, it might be an incomplete download. To check, we ask the server for the size of the file with a HEAD request (which is almost instant, quicker than downloading it.) and compare that to the size of the file on disk. If it's the same, do not re-download the file.

We get throttled a lot, which appears as 403 errors. Sleeping and retrying fixes that. 

In [ ]:
# create raw folder if it doesn't exist
if not os.path.exists(raw_files_path):
    os.makedirs(raw_files_path)

In [ ]:
# check there are no duplicate files in different locations on the website

# deduplicate based on everything after the last slash
urls = list({u.split('/')[-1]:u for u in urls}.values())
len(urls)

In [ ]:

# shuffle, so that large files aren't clumped together.
# this makes the progress bar estimate more accurate (compared to downloading all the big files last/first)
# and if we don't download all the small files consecutively, we're less likely to be throttled
shuffle(urls) # mutates list in place

In [ ]:


# using urllib3 instead of requests
# because it's better for streaming large files to disk
# https://stackoverflow.com/a/62075390/5443120
http = urllib3.PoolManager(retries=8)


In [ ]:

def get_remote_size(url, retries=6):
    r = http.request('HEAD', url)
    if r.status >= 300:
        if retries > 0:
            logger.warning(f"Retrying after bad status ({r.status}) for HEAD {url}")
            sleep(randrange(5))
            return get_remote_size(url, retries=retries-1)
        else:
            logger.error(f"Not retrying after bad status ({r.status}) for HEAD {url}")
            raise ValueError(f"bad status ({r.status}) for HEAD {url}")
    return int(r.headers['Content-Length'])

def already_downloaded(url, local):
    if not os.path.exists(local):
        return False
    else:
        return get_remote_size(url) == os.path.getsize(local)

def download(url, retries=6):
    fname = url.split('/')[-1]
    local_path = os.path.join(raw_files_path, fname)
    try:
        if already_downloaded(url, local_path):
            logger.info(f"Skipping already downloaded {url}")
            return 'skipped'
        else:
            with open(local_path, 'wb') as f:
                r = http.request('GET', url, preload_content=False)
                if r.status >= 300:
                    logger.warning(f"Retrying after bad status ({r.status}) for GET {url}")
                    sleep(randrange(5))
                    return download(url, retries=retries-1)
                logger.info(f"Downloading {url}")
                shutil.copyfileobj(r, f)
                logger.info(f"Downloaded {url}\n")
    except (urllib3.exceptions.HTTPError, ValueError) as ex:
        # tidy up partial download
        try:
            os.remove(local_path)
        except OSError:
            pass
        if retries <= 0:
            logger.error(f"Not retrying after error with {url}: {ex}")
            return ex
        else:
            logger.warning(f"Retrying after error with {url}: {ex}")
            sleep(randrange(3))
            return download(url, retries=retries-1)
    return 'ok'
                


if __name__ == '__main__':
    
    # use multiprocessing to process the files concurrently
    # tqdm for a progress bar
    # list(tqdm(imap())) thing explained here: https://stackoverflow.com/a/41921948/5443120
    if use_multiprocessing:
        with Pool(2*os.cpu_count()) as p:
            statuses = list(tqdm(p.imap(download, urls), total=len(urls)))
    else:
        statuses = [download(url) for url in tqdm(urls)]

    # if these are only HEAD 404s for a few files, that's ok. They've probably moved from /CURRENT to /ARCHIVE (under a different file name)
    # re-download later and you'll get them
    assert all(s in ['ok', 'skipped'] for s in statuses), "some files failed to be downloaded."

## Unzip

We list all files in the directory, instead of getting the list of files from earlier. This is because we may want to re-run the playbook from this point, without re-downloading the files.

In [ ]:
# create raw folder if it doesn't exist
for d in [unzipped_files_path, unzipping_temp_path]:
    if not os.path.exists(d):
        os.makedirs(d)

In [ ]:
logger.reset()

# takes a CSV
# fname is the name of the CSV
# with f being an optional virtual file-like object inside an unzipped zip
# we write to a random filename, then move to the destination
# so that if two processes are unzipping different files to the same destination concurrently, they won't corrupt the file.
def process_csv(fname, f=None):
    assert fname.lower().endswith('.csv')
    temp_path = os.path.join(unzipping_temp_path, str(uuid4())) # random file within that folder
    dest = os.path.join(unzipped_files_path, fname.split('/')[-1])
    if not dest.lower().endswith('.gz'):
        dest = dest + '.gz'
        
    if f is None:
        with open(fname, 'rb') as f:
            process_csv(fname, f)
    else:
        with gzip.open(temp_path, 'wb', compresslevel=compresslevel) as out:
            shutil.copyfileobj(f, out)
        os.rename(temp_path, dest)
        
def process_zip(fname, f=None):
    if f is None:
        with open(fname, 'rb') as f:
            process_zip(fname, f)
    else:
        #print(f"Processing {fname}")
        with ZipFile(f) as z:
            for m in z.namelist():
                if m.lower().endswith('.zip'):
                    with z.open(m, mode='r') as mf:
                        process_zip(m, mf)
                elif m.lower().endswith('.csv'):
                    with z.open(m, mode='r') as mf:
                        process_csv(m, mf)
                else:
                    logger.warning(f"Found unknown file {m} in zip {fname}")


def process(path):
    if path.lower().endswith('.csv'):
        process_csv(path)
    else:
        process_zip(path)

# call process()
# catch (almost) all errors, and return them
# then after we process all files, check these errors
def _process(path):
    try:
        process(path)
    except KeyboardInterrupt:
        raise
    except Exception as ex: # catch anything except keyboard interrupt
        logger.error(f"Failed to unzip {path}: {ex}")
        print(f"Failed to unzip {path}: {ex}")
        return ex # not rethrown
        
# tqdm is used to show a progress bar
# If you want to add multiprocessing here, watch out for file clobbering
# multiple source zip files may contain duplicates of the same destination .csv
# if you try to write to the dest file twice concurrently, you'll get corrupted files, but may not notice
# For a given .csv filename, the content is always the same if you find it in different source zips

if __name__ == '__main__':
    files = [os.path.join(raw_files_path, p) for p in os.listdir(raw_files_path)]

    # shuffle file order
    # so that we don't do all the small files first
    # then all the large files last 
    # (or the other way around)
    # this makes the progress bar more accurate
    shuffle(files) # mutates list in place
    
    if use_multiprocessing:
        if leave_unused_cpu:
            num_processes = os.cpu_count() - 2 # *2 because we assume hyperthreading
        else:
            num_processes = os.cpu_count()
        with Pool(num_processes) as p:
            statuses = list(tqdm(p.imap(_process, files), total=len(files)))
    else:
        statuses = [_process(file) for file in tqdm(files)]
    assert all(s is None for s in statuses), "some files failed to be unzipped"
    

## Convert Files

After unzipping files, we're left with what look like a bunch of CSVs. (Or `.csv.gz`, which I'm hoping excel can open.) But if you open them up you'll see they're not a normal CSV. They're a concatenation of many CSV files into one. (e.g. note how the number of columns changes as you scroll down.) So we want to unconcatenate them, or "split" them. So each ".CSV" file becomes many ".CSV" files.

To understand this format, read `Guide to CSV Data Format Standard.pdf`, which is saved adjacent to this file. Available [here](https://web.archive.org/web/20230414160249/https://aemo.com.au/-/media/files/market-it-systems/guide-to-csv-data-format-standard.pdf?la=en).

To read these files, look at the first column, which is C, D, or I.

* `C` means this is metadata. Typically the first and last row of the file. The first row tells you things like when the file was generated. We can probably ignore this. (Other timestamps are inside the data itself.)
* `I` means this is the header row for a new table
* `D` means this is a data row for the same table as the previous row

In the `D` and `I` rows, the second and third column contain info about which table this data belongs to (`REPORT_NAME` and `REPORT_SUBTYPE`). This is described more below. For now we put output files under a 2-layer folder structure based on this. 

The fourth column is an integer related to some kind of schema versioning. It's really nuanced. I don't think this will matter for our purposes. 

Note that pandas cannot read these files (because the datatype changes across rows within the same file, and the number of rows changes.) So we just read line-by-line as text. Once it's split, at a later stage we'll read with Pandas (or R).

In [ ]:
# I've found some particular files have funny encodings
# just ignore them. We don't need them for our analysis.
csvgz_files_to_ignore = [
    # regex patterns
    f"7_days_High_Impact_Outages_\d+.csv.gz"
]

In [ ]:
logger.reset()

# create folder if it doesn't exist
if not os.path.exists(split_unmapped_files_path):
    os.makedirs(split_unmapped_files_path)

# call split
# catch (almost) all errors and return them
# so that if one file fails, we still process the others
# and then check errors all at the end.
def _split(fname):
    try:
        split(fname)
    except KeyboardInterrupt:
        raise
    except Exception as ex:
        if any(re.search(ptn, fname) for ptn in csvgz_files_to_ignore):
            logger.warning(f"Failed to split {fname}: {ex}, but ignoring this particular file")
        else:
            logger.error(f"Failed to split {fname}: {ex}")
            return ex

def split(fname):
    source_path = os.path.join(unzipped_files_path, fname)

    if fname.lower().endswith('.gz'):
        with gzip.open(source_path, 'rt', newline='') as f:
            split_f(f, fname)
    else:
        assert fname.lower().endswith('.csv'), f"Unsure how to open {fname}"
        with open(source_path, 'r', newline='') as f:
            split_f(f, fname)
        

def split_f(src, fname):
    csv_r = csv.reader(src)
    first_row = next(csv_r)
    expected_start = 'C,SETP.WORLD'
    if first_row[0] != 'C':
        # some other files end up in the dataset
        # e.g. int668_v1_schedule_log_rpt_1~20231124133149.csv.gz
        # just ignore files like that
        logger.warning(f"First line does not start with C, {first_row[:10]=} in {fname}. Ignoring")
        return

    row = next(csv_r)

    # these will be set later
    dst = None
    chars_to_skip = None

    try:
        while True:
            if row[0] == 'C':
                # 2nd C line, which should be the last line of the file
                if row[1] == 'END OF REPORT':
                    checksum = row[-1]
                    assert int(checksum) == csv_r.line_num, f"Checksum on last line doesn't match in {fname}"
        
                    # close off the last output file
                    assert dst is not None
                    dst = None 
                    fout.close()
                    
                    assert src.read().strip() == ''

                    if num_d_rows == 0:
                        logger.warning(f"No data rows, only header row written for {fname}, so deleting output file")
                        os.remove(dest_path)
                    
                    break
                elif row[1] == 'SETTLEMENTS RESIDUE CONTRACT REPORT':
                    logger.info(f"Ignoring C line {csv_r.line_num} in {fname}: {row[1]}")
                else:
                    raise ValueError(f"Unexpected C line {csv_r.line_num} in {fname}: {row}")
            elif row[0] == 'I':
                # start of new file
                row_type, report_name, report_subtype, version, *remainder = row
                cols_to_skip = 4
    
                if report_name in ['', None]:
                    report_name = REPORT_NAME_NULL_PLACEHOLDER
                if report_subtype in ['', None]:
                    report_subtype = REPORT_SUBTYPE_NULL_PLACEHOLDER
                
                # create destination folder if it doesn't exist
                dest_folder = os.path.join(split_unmapped_files_path, report_name, report_subtype)
                if not os.path.exists(dest_folder):
                    try:
                        os.makedirs(dest_folder)
                    except FileExistsError:
                        # race condition
                        pass
                
                dest_path = os.path.join(dest_folder, fname)
                fout = gzip.open(dest_path, 'wt', compresslevel=compresslevel)
                dst = csv.writer(fout)
                dst.writerow(remainder)
                num_d_rows = 0
    
            else:
                assert row[0] == 'D'
                dst.writerow(row[cols_to_skip:])
                num_d_rows += 1
    
            row = next(csv_r)
    except Exception as e:
        logger.error(f"Error processing {fname}: {e}")
        # tidy up partial write
        try:
            fout.close()
        except (NameError, OSError):
            pass
        try:
            os.remove(dest_path)
        except (NameError, OSError):
            pass
        raise # rethrow original error
                
if __name__ == '__main__':
    files = os.listdir(unzipped_files_path)

    # shuffle file order
    # so that we don't do all the small files first
    # then all the large files last 
    # (or the other way around)
    # this makes the progress bar more accurate
    # it also means that we process the first file of each type sooner
    # so that if there's issues, we can spot them in the logs sooner (e.g. 20 minutes in, instead of 1h in.)
    shuffle(files) # mutates list in place
    
    if use_multiprocessing:
        if leave_unused_cpu:
            num_processes = os.cpu_count() - 2 # *2 because we assume hyperthreading
        else:
            num_processes = os.cpu_count()
        with Pool(num_processes) as p:
            statuses = list(tqdm(p.imap(_split, files), total=len(files)))
    else:
        statuses = [_split(file) for file in tqdm(files)]
    
    [(s,f) for (s,f) in zip(statuses, files) if s is not None][:10]
    assert all(s is None for s in statuses), "some files failed to be split"
    

## Map each file to a table

We now look at the REPORT_TYPE and REPORT_SUBTYPE to figure out which table each one belongs to.

Sometimes it's obvious. E.g. `DISPATCH` and `PRICE` belongs to the `DISPATCHPRICE` table. But sometimes it's not obvious. (And sometimes there are blanks.) The best way I've found is to look up those two values in a particular file. That file is no longer on the internet. But we can find an older version [here](https://web.archive.org/web/20220812083311/https://visualisations.aemo.com.au/aemo/di-help/Content/Data_Model/MMS_Data_Model.htm). (That's for schema version 5.1. We're up to 5.2 now. But the differences are small enough that it shouldn't be an issue. We care about so few tables that we could probably hard-code this mapping.) 

Here we just move the files on disk from one folder to another. We don't copy and re-write them.

We could have done this step when we originally wrote the files. But I deliberately split it out to make it clearer what's happening.

In [ ]:
metadata_df = pd.read_excel(metadata_path, sheet_name=sheet)
metadata_df[report_name_column].fillna(REPORT_NAME_NULL_PLACEHOLDER, inplace=True)
metadata_df[report_subtype_column].fillna(REPORT_SUBTYPE_NULL_PLACEHOLDER, inplace=True)

# there are some packages I do not know how to map to tables

packages_to_ignore = [
    # maybe there's a table for these,
    # or they're legacy tables
    # these are ones we don't care about
    ('DINT', REPORT_SUBTYPE_NULL_PLACEHOLDER),
    ('TINT', REPORT_SUBTYPE_NULL_PLACEHOLDER),
    ('DCONS', REPORT_SUBTYPE_NULL_PLACEHOLDER),
    ('DREGION', REPORT_SUBTYPE_NULL_PLACEHOLDER),
    ('SPDCPC', REPORT_SUBTYPE_NULL_PLACEHOLDER),
    ('SRAFINANCIALS', 'RECONCILIATION_SUMMARY'),
    ('DAILY', 'MLF'), # electrical transmission loss factors.
    ('BILLING_CONFIG', 'BILLSMELTERRATE'), # alumninium smelter info. Possibly relevant? (Smeltering makes up 30% of NSW load)
    ('BILLING', 'CSP_SUPPORTDATA_SRA'),
    ('BILLING', 'ASPAYMENT_SUMMARY'), # probably belongs to BILLINGASPAYMENTS table, but this is not relevant to us so I haven't bothered checking
    ('BILLING', 'DIRECTION_CRA'),
    ('TRADING', 'CUMULATIVE_PRICE'),
    ('TREGION', 'NULL'),
    ('DAILY', 'WDR_NO_SCADA'),
    ('METER_DATA', 'GEN_DUID'),
    ('SEVENDAYOUTLOOK', 'PEAK'),
    ('TUNIT', 'NULL'),
    ('GPG', 'MARKET_SUMMARY'),
    ('GPG', 'CASESOLUTION'),
    ('GPG', 'CONSTRAINTSOLUTION'),
    ('GPG', 'PRICESOLUTION'),
    ('GPG', 'INTERCONNECTORSOLUTION'),
    ('CAUSER_PAYS_SCADA', 'NETWORK'),
    ('PDR_REPORT', 'COLUMN'),
    ('DUNIT', 'NULL'),
    ('YESTBID', 'BIDDAYOFFER'),
    ('YESTBID', 'BIDPEROFFER'),
    ('RESIDUE_PRICE_OFFER', 'NULL'),
    ('RESIDUE_PRICE_BID', 'NULL'), # columns don't match the RESIDUE_PRICE_BID table, almost match RESIDUE_PRICE_FUNDS_BID
    ('IBEI', 'PUBLISHING'),
    ('EMSLIMITS', 'LIM_ALTLIM'),
    ('DEMAND', 'HISTORIC'),

    
    ('PDR_REPORT', 'TABLE'),    # manifest metadata
    ('PDR_REPORT', 'FILE'),    # manifest metadata
    ('PDR_REPORT', 'MAPPING'),    # manifest metadata
    ('PDR_REPORT', 'COLUMN'),    # manifest metadata

    # Gas data
    ('GSH', 'PARTICIPANT_OPSTATE'),
    ('GSH', 'PARTICIPANTS'),
    ('GSH', 'AUCTION_CURTAILMENT_NOTICE'),
    ('GSH', 'AUCTION_PRICE_VOLUME'),
    ('GSH', 'BENCHMARK_PRICE'),
    ('GSH', 'PARK_SERVICES'),
    ('GSH', 'AUCTION_QUANTITIES'),
    ('GSH', 'FACILITIES'),
    ('GSH', 'TRANSACTION_SUMMARY'),
    ('GSH', 'HISTORICAL_SUMMARY'),
    ('GSH', 'NOTIONAL_POINTS'),
    ('GSH', 'REVISED_AUCTION_QUANTITIES'),
    ('GSH', 'PIPELINE_SEGMENTS'),
    ('GSH', 'CAPACITY_TRANSACTION'),
    ('GSH', 'ZONES'),
    ('GSH', 'SERVICE_POINTS'),

    # we should maybe double check
    ('FORCE_MAJEURE', 'MARKETSUSREGION'),
    ('FORCE_MAJEURE', 'MARKETSUSPENSION')

]


# there are some packages which aren't in the spreadsheet, but we can guess them
# in particular there's a CO2 one I can't 
package_exceptions = {
    ('DISPATCH', 'CASESOLUTION'): 'DISPATCHCASESOLUTION',
    ('GENCONSETTRK', REPORT_SUBTYPE_NULL_PLACEHOLDER): 'GENCONSETTRK',
    ('DISPATCH', 'REGIONFCASREQUIREMENT'): 'DISPATCH_FCAS_REQ',

    # this one is possibly useful
    # example file is CO2EII_AVAILABLE_GENERATORS.CSV.gz
    # It looks like it might go into BILLING_CO2E_PUBLICATION, but the columns are different
    # Example:
    #STATIONNAME,DUID,GENSETID,REGIONID,CO2E_EMISSIONS_FACTOR,CO2E_ENERGY_SOURCE,CO2E_DATA_SOURCE
    #"Appin Power Plant",APPIN,APPIN,NSW1,0.56318004,"Coal seam methane",NGA2022
    # so define a new table for it
    ('CO2EII', 'PUBLISHING'): 'CO2EII_AVAILABLE_GENERATORS',


}

def map_table(report_name, report_subtype):
    candidates = metadata_df.loc[(metadata_df[report_name_column] == report_name) & (metadata_df[report_subtype_column] == report_subtype), table_name_column]
    # sometimes there's duplicates
    # but assert they're all the same answer
    tables = set(candidates)
    if len(tables) == 0 and (report_name, report_subtype) in package_exceptions:
        return package_exceptions[(report_name, report_subtype)]
        
    assert len(tables) > 0, f"Unable to map {report_name=} {report_subtype=} to a table name. If you don't need this table, add ('{report_name}', '{report_subtype}') to packages_to_ignore"
    assert len(tables) == 1, f"Multiple target tables found for  {report_name=} {report_subtype=}: {tables}"
    table = tables.pop() # choose the only one
    return table

# unit testing
assert map_table('DISPATCH', 'PRICE') == 'DISPATCHPRICE'

In [ ]:
# don't bother with multiprocessing for this
# moving a folder of N files is O(1) not O(N)
unmappable = []
for report_name in tqdm(os.listdir(split_unmapped_files_path)):
    subdir = os.path.join(split_unmapped_files_path, report_name)
    for report_subtype in os.listdir(subdir):
        subsubdir = os.path.join(subdir, report_subtype)
        
        if (report_name, report_subtype) in packages_to_ignore:
            # leave these files where they are
            continue
        if len(os.listdir(subsubdir)) == 0:
            # empty folder, don't bother mapping
            continue
        try:
            table = map_table(report_name, report_subtype)
        except AssertionError:
            unmappable.append((report_name, report_subtype))
        else:
            # note that multiple source directories may map to the same destination directory
            # so we move files one at a time, instead of moving the whole directory and erasing what's there
            for filename in os.listdir(subsubdir):
                source_path = os.path.join(subsubdir, filename)
                dest_dir = os.path.join(split_mapped_files_path, table)
                if not os.path.exists(dest_dir):
                    os.makedirs(dest_dir)
                dest_path = os.path.join(dest_dir, filename)
                shutil.move(source_path, dest_path)
assert len(unmappable) == 0, f"Unable to map some files to tables. Details in unmappable"

## Convert many CSV to Parquet

We have many files per folder. We want to combine into just one parquet file. Some are too large to fit in memory.
So we stream the files when converting, using Arrow. (We don't use pandas because AEMO data has missing values in columns which are integers, datetimes etc. Pandas doesn't like that.) 

But Arrow throws an error when combining one file with an integer column, and another file with double/float/decimal in the same column. It's too dumb to merge those automatically. So we need to tell it explicitly the data type of decimal columns.

Note that AEMO's schemas change over time. New columns get added over the years. So a particular column might be missing in some files. And the column ordering is not consistent.

Note also that some files have no value in a certain column for the first million rows, then a value appears. Pyarrow and pandas both check only the first few rows to detect datatypes in CSV files. 

Note that some cells can be like `-7E-05`. So there's a letter character, but it's a float.

Note that there's too many datasets to explicitly hard-code each column type. (And AEMO does not offer a machine-readable version. Unless you use their proprietary software that's complex and not available to researchers.) So we want an automatic way of detecting data types.

So our solution here is:

1. We go through each file in a folder, find the union of all the column names.
2. We read through each file, checking each column, each row. We want to keep track of columns which are truly integers everywhere, vs ones which have at least one float (number with a decimal point). We also keep track of what's a string, bool etc. This is using out custom `SchemaType` type. For this step we tell pandas that all columns are string type, and we try to y p e t to int/float etc, and see if there's an error. We do this with pandas, batching large files.
3. For each folder (set of files that belong in the same final file) we call pyarrow, point it at the folder of input CSVs, and tell it to write a single output CSV. We tell pyarrow explicitly what datatypes of each column are. For timestamps, pyarrow can't read timestamps from CSV. So we tell pyarrow to read them as a string, and then we convert to a datetime in a step before writing. This is a bit fiddle because we're chunking the file into batches. Note that handling the timezone is a bit tricky.
4. We confirm that the timezone was handled correctly by unit-testing a specific value in a final parquet file.

We could also group by some columns. e.g. to write one parquet file per region. This might improve query performance later. But we haven't done this yet.

In [ ]:
if not os.path.exists(one_csv_per_table_path):
    os.makedirs(one_csv_per_table_path)

In [ ]:
# the next cell will take a long time to run
# and uses lots of CPU
# If you want to free up CPU to use your laptop for something else,
# without losing progress,
# create this file next to this ipynb file
pause_path = 'pause.txt'
def check_for_pause():
    if os.path.exists(pause_path):
        logger.info("paused")
        while os.path.exists(pause_path):
            sleep(5)
        logger.info("Resumed")

check_for_pause()

In [ ]:
# as we iterate through the data row by row
# this object keeps track of the datatypes we've seen for a particular column
class SchemaType:
    # start with all attributes True
    # i.e. the corresponding column could be any datatype
    # When we see a value not compatible with any of these datatypes
    # set it to False
    empty = True # are all values in the column empty?
    bool = True # a.k.a. binary
    int = True
    decimal = True # a.k.a. float/double
    datetime = True
    time = True # time without date
    string = True

    # if it's an int, what's the min and max values seen?
    # to figure out int32 vs int64 vs uint32
    max_int = None
    min_int = None

    # take in a numpy array (column of a pandas dataframe)
    # which is a string (that may have null values)
    # and check whether it's incomptible with each datatype
    # this will be called many times per column
    def update(self, vals: np.array):
        # ignore empty values
        vals = vals.dropna()
        if len(vals) == 0:
            return
        else:
            self.empty = False

        # For booleans, only 1 and 0 is in AEMO data. Not T, true, Y etc
        # but watch out. Some data is "01". We don't want to count that as boolean.
        # (Arrow won't convert that later.)
        # note that vals is an array of strings (of int or whatever)
        if self.bool:
            self.bool &= vals.isin(['0', '1']).all()
        
        if self.int:
            try:
                i = vals.astype(int)
            except:
                self.int = False
            else:
                # it is an int. Update min/max values for this column
                if self.max_int is None:
                    self.max_int = i.max()
                    self.min_int = i.min()
                else:
                    self.max_int = max(i.max(), self.max_int)
                    self.min_int = min(i.min(), self.min_int)


        # not elif, since we want to go here if we've just set self.int to False after `if self.int`
        if self.decimal and (not self.int):
            # remember that floats can be like '1e2', so not just digits and a dot
            try:
                vals.astype(float)
            except ValueError:
                self.decimal = False

        if self.datetime:
            # e.g. 2023/10/01 00:00:00
            # not using strptime yet. We just want to check that it is a datetime.
            # regex match is faster than strptime
            matches = vals.str.match(r"\d{4}/\d{1,2}/\d{1,2} \d{1,2}:\d{1,2}:\d{1,2}")
            if not matches.all():
                self.datetime = False
                
        if self.time:
            # e.g. 12:34:56
            matches = vals.str.match(r"\d{1,2}:\d{1,2}:\d{1,2}")
            if not matches.all():
                self.time = False

    # return a datatype, as a pyarrow datatype
    # which is not the same as a normal python datatype
    # e.g. pyarrow.bool_() instead of just bool
    def get_type(self) -> pyarrow.DataType:
        if self.empty:
            # all rows have an empty value
            # choose anything
            logger.info(f"Col is empty")
            return pyarrow.string()
        elif self.bool:
            return pyarrow.bool_()
        elif self.int:
            # it's an integer, not decimal
            # but is it signed/unsigned, and how many bits?
            assert self.min_int is not None
            assert self.max_int is not None
            if self.min_int >= 0:
                # unsigned
                _type = pyarrow.int64()
                for bits in [32, 16, 8]:
                    if self.max_int <= 2**bits - 1:
                        _type = getattr(pyarrow, f"uint{bits}")()
            else:
                # signed
                _type = pyarrow.int64()
                for bits in [32, 16, 8]:
                    if (self.max_int <= 2**(bits-1) - 1) and (self.min_int >= -2**(bits-1) ):
                        _type = getattr(pyarrow, f"int{bits}")()
            return _type
        elif self.decimal:
            # we could differentiate between 32 and 64. I can't be bothered.
            # I think R treats everything as 64 bits anyway
            return pyarrow.float64()
        else:
            # includes arbitrary strings, as well as datetime
            # datetime will be parsed from CSV as string
            # and then converted to datetime/time as a second step afterwards
            return pyarrow.string()

# takes in the name of a table (one folder of CSVs)
# returns a dictionary of column name to SchemaType
def detect_schema(table) -> Dict[str, SchemaType]:
    source_dir = os.path.join(split_mapped_files_path, table)

    # first loop through each file, read just the first row to get the header row
    logger.info(f"Checking headers in {table}")
    schema = {}
    
    for file in os.listdir(source_dir):
        check_for_pause()
        path = os.path.join(source_dir, file)
        assert path.lower().endswith('.csv.gz')

        for df in pd.read_csv(path, chunksize=100000, dtype=str):
            t = pyarrow.Table.from_pandas(df)
            for col in df:
                if col not in schema:
                    schema[col] = SchemaType()
                schema[col].update(df[col])
    
    return schema

# catch errors for multiprocessing
# so if one fails, the others keep going
def _detect_schema(table):
    try:
        schema = detect_schema(table)
    except Exception as ex:
        logger.exception(f"Failed to process {table}")
        logger.error(ex)
        return [None, ex]
    else:
        return [schema, None]
        
if __name__ == '__main__':
    folders = os.listdir(split_mapped_files_path)
    
    if use_multiprocessing:
        if leave_unused_cpu:
            num_processes = os.cpu_count() - 2 # *2 because we assume hyperthreading
        else:
            num_processes = os.cpu_count()
        with Pool(num_processes) as p:
            result = list(tqdm(p.imap(detect_schema, folders), total=len(folders)))
    else:
        result = [detect_schema(folder) for folder in tqdm(folders)]


    schemas = dict(zip(folders, result))

In [ ]:
# Save the result to a file, for debugging purposes
schema_pickle_path = os.path.join(base_data_dir, 'schemas.pickle')
with open(schema_pickle_path, 'wb') as f:
    pickle.dump({'test': schemas}, f)

In [ ]:
if not os.path.exists(one_parquet_per_table_path):
    os.makedirs(one_parquet_per_table_path)

In [ ]:
# takes in the name of a folder of CSVs
# converts them all to a single parquet file
def convert_csv_parquet(table):
    csv_folder = os.path.join(split_mapped_files_path, table)
    parquet_file = os.path.join(one_parquet_per_table_path, table + '.parquet')

    input_schema = {}
    output_schema = {}
    datetime_columns = []
    column_names = []
    for (c,s) in schemas[table].items():
        input_schema[c] = s.get_type()
        if s.datetime:
            datetime_columns.append(c)
            output_schema[c] = pyarrow.timestamp('s', tz=f'+{TIMEZONE_OFFSET:02d}:00')
        else:
            output_schema[c] = s.get_type()
        column_names.append(c)
    input_schema = pa.schema(input_schema)
    output_schema = pa.schema(output_schema)
            
    
    with pyarrow.parquet.ParquetWriter(parquet_file, output_schema) as writer:
        for csv_file in os.listdir(csv_folder):
            try:
                csv_path = os.path.join(csv_folder, csv_file)
                csv_reader = pyarrow.csv.open_csv(csv_path, 
                                                  convert_options=pyarrow.csv.ConvertOptions(
                                                      column_types=input_schema, 
                                                      include_missing_columns=True, 
                                                      include_columns=column_names))
    
                for batch in csv_reader:
                    check_for_pause()
                    columns = []
                    if datetime_columns == []:
                        # table has no datetime columns
                        # just write straight into the parquet file
                        writer.write(batch)
                    else:
                        # parse some columns from string to datetime
                        
                        for (i, column_name) in enumerate(column_names):
                            assert isinstance(column_name, str), f"column_name is type {type(column_name)}"
                            column_data = batch[column_name]
                            if column_name in datetime_columns:
                                # pyarrow assumes the timesstamp is UTC by default
                                # the schema specifies the timezone as UTC+10 (which we want)
                                # but it adds 10h when doing the conversion
                                # so let's subtract 10h
                                # This is checked with a unit test later
                                column_data = pc.strptime(column_data, format="%Y/%m/%d %H:%M:%S", unit="s", error_is_null=True)
                                column_data = pc.add(column_data, -dt.timedelta(hours=TIMEZONE_OFFSET))
                            columns.append(column_data)
                        
                        updated_table = pyarrow.Table.from_arrays(
                            columns, 
                            schema=output_schema
                        )
                        writer.write(updated_table)
            except Exception:
                # additional logging
                logger.error(f"Error with file {csv_path}")
                raise


if __name__ == '__main__':
    folders = os.listdir(split_mapped_files_path)
    
    if use_multiprocessing:
        if leave_unused_cpu:
            num_processes = os.cpu_count() - 2 # *2 because we assume hyperthreading
        else:
            num_processes = os.cpu_count()
        with Pool(num_processes) as p:
            list(tqdm(p.imap(convert_csv_parquet, folders), total=len(folders)))
    else:
        for folder in tqdm(folders):
            convert_csv_parquet(folder)

In [ ]:
# Unit test a specific value
table = 'DISPATCHPRICE'
path = parquet_file = os.path.join(one_parquet_per_table_path, table + '.parquet')
df = pd.read_parquet(path)

# check the first row of PUBLIC_DISPATCHIS_201708091630_0000000286225673.CSV
utc_offset = dt.timedelta(hours=TIMEZONE_OFFSET)
tz = dt.timezone(utc_offset)
t = dt.datetime(year=2017, month=8, day=9, hour=16, minute=30, tzinfo=tz)

# I manually looked in PUBLIC_DISPATCHIS_201708091630_0000000286225673.CSV
# at the first row.
expected_rrp = 78.47404
vals = df.loc[(df['SETTLEMENTDATE'] == t) & (df['REGIONID'] == 'SA1'), 'RRP']
assert len(vals) == 1
assert (vals == expected_rrp).all()